# Convolutional Neural Network

We will use a CNN to recognize handwritten digits from the MNIST dataset. 

CNNs are well-suited to image recognition tasks because they learn patterns that are invariant under translations. This means that it it learns a pattern in one part of the image, then it can recognize the same pattern in other parts of the image.

We begin by importing the various modules and functions that we will use in the noteboook.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)
from keras.datasets import mnist
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten
from keras.layers.core import Dense, Dropout, Activation

/Users/davidradcliffe/anaconda/envs/deeplearning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Load the training and test data for the MNIST dataset. We need to reshape the images from (28, 28) to (28, 28, 1) because the convolution layers assume that images have color channels. Our images are grey scale, so they have only one color channel.

The values are rescaled to range from 0 to 1, instead of 0 to 255, because it is difficult to train a neural network when the inputs are large.

The target values (y_train and y_test) must be one-hot encoded.

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1) / 255
X_test = X_test.reshape(-1, 28, 28, 1) / 255
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

We construct a neural network with two convolutional layers and a dense hidden layer.
Each convolution is followed by a MaxPooling layer, which reduces the data and helps the network to learn bigger patterns. A dropout layer is added to reduce overfitting.

After the convolution layers, we add a dense hidden layer with 128 nodes. We must flatten the output of a convolutional layer in order to connect it to a dense layer. Flattening means that the 4-dimensional tensor is reshaped into a 2-d array (or matrix).

Since this is a classification task with 10 classes, the output layer has 10 nodes with a softmax activation.

In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               102528    
__________

Now we compile the model and fit it on the training data.

In [6]:
model.fit(X_train, y_train, epochs=4, verbose=1, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/4
60000/60000 [==============================] - 58s 961us/step - loss: 0.1554 - acc: 0.9520 - val_loss: 0.0535 - val_acc: 0.9819
Epoch 2/4
60000/60000 [==============================] - 60s 993us/step - loss: 0.0627 - acc: 0.9811 - val_loss: 0.0629 - val_acc: 0.9819
Epoch 3/4
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0488 - acc: 0.9851 - val_loss: 0.0308 - val_acc: 0.9897
Epoch 4/4
60000/60000 [==============================] - 55s 909us/step - loss: 0.0417 - acc: 0.9876 - val_loss: 0.0314 - val_acc: 0.9902


In [7]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 3s 314us/step


[0.03143580365489979, 0.9902]

We achieved 99% accuracy on the test set.